# 2. CLEANSED LAYER

In [1]:
!pip install pandasql

"pip" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [2]:
import pandas as pd
import pandasql as psql

ModuleNotFoundError: No module named 'pandas'

## OF 123456.xlsx

In [92]:
path_raw = 'C:\\Users\\anton\\Datathon\\01_Raw\\'
path_cleansed = 'C:\\Users\\anton\\Datathon\\02_Cleansed\\'

# Leer el archivo Excel y guardar el contenido en un DataFrame
df = pd.read_excel(f'{path_raw}OF 123456.xlsx')

# Escribir una consulta SQL sobre el DataFrame
consulta = """

    SELECT 
    CAST(REPLACE(`Lote`, '/', '') AS INTEGER) AS Lote, 
    Orden AS OF,
    `Cantidad entregada` AS Cantidad_entregada

    FROM df
    """

In [93]:
# Ejecutar la consulta
resultado = psql.sqldf(consulta, locals())
print(resultado.dtypes)
# Guardar el resultado en un archivo Excel
resultado.to_excel(f'{path_cleansed}OF_123456.xlsx', index=False)

# Mostrar el resultado
print(resultado)


Lote                    int64
OF                      int64
Cantidad_entregada    float64
dtype: object
      Lote         OF  Cantidad_entregada
0    23019  200178572               13.80
1    23020  200179217               13.60
2    23021  200181620               13.50
3    23022  200182428               13.80
4    23023  200182429               13.70
..     ...        ...                 ...
202  24108  200205686               13.63
203  24106  200205695               13.69
204  24107  200205696               13.45
205  24109  200205687               11.35
206  24110  200205688               13.77

[207 rows x 3 columns]


## Cinéticos IPC

### Cultivos finales

In [80]:
import pandas as pd
import pandasql as ps

# Definir las rutas de las carpetas
path_raw = 'C:\\Users\\anton\\Datathon\\01_Raw\\'
path_cleansed = 'C:\\Users\\anton\\Datathon\\02_Cleansed\\'

# Leer el archivo Excel y guardar el contenido en un DataFrame
df = pd.read_excel(f'{path_raw}Cinéticos IPC.xlsx', sheet_name='Cultivos finales')

# Mostrar los tipos de datos del DataFrame
print(df.dtypes)

# Escribir una consulta SQL sobre el DataFrame
consulta = """
    SELECT 
        CAST(Lote AS Integer) AS Lote,
        Fecha,
        CAST(NULLIF(Turbidez, '-') AS Float) AS Turbidez,
        Viabilidad,
        CASE
            WHEN `Glucosa g/L` REGEXP '^[0-9]+\\.[0-9]+$' THEN CAST(`Glucosa g/L` AS Float)
            ELSE NULL
        END AS Glucosa
    FROM df
"""
resultado = psql.sqldf(consulta, locals())
resultado['Fecha'] = pd.to_datetime(resultado['Fecha'])

Lote                   object
Fecha          datetime64[ns]
Turbidez               object
Viabilidad            float64
Glucosa g/L            object
dtype: object


In [82]:
# Guardar el resultado en un archivo Excel
resultado.to_excel(f'{path_cleansed}CineticosIPC_Cultivos.xlsx', index=False)

# Mostrar el resultado
print(resultado)

Lote                   int64
Fecha         datetime64[ns]
Turbidez             float64
Viabilidad           float64
Glucosa              float64
dtype: object
       Lote               Fecha  Turbidez    Viabilidad  Glucosa
0     23019 2023-03-21 07:30:00     17.28  8.333333e+06    1.744
1     23019 2023-03-21 11:00:00     17.28  1.397333e+07    1.968
2     23019 2023-03-21 14:00:00     18.40  1.688000e+07    1.920
3     23019 2023-03-21 18:00:00     19.76  2.474667e+07    1.880
4     23019 2023-03-22 07:00:00     39.12  1.000000e+08    1.568
...     ...                 ...       ...           ...      ...
1906  24110 2024-07-04 15:30:00     58.40  8.053333e+07    1.584
1907  24110 2024-07-04 19:30:00     69.76  1.021333e+08    1.248
1908  24110 2024-07-05 06:30:00    100.80  1.525333e+08    0.600
1909  24110 2024-07-05 08:00:00    104.80  1.514667e+08    0.520
1910  24110 2024-07-05 10:10:00    104.80  1.573333e+08    0.592

[1911 rows x 5 columns]
